# Обучение поискового индекса

За основу поиска кандидатов будет взят ANN-алгоритм из библиотеки hnswlib.

В индекс будут положены вектора документо - наименований товаров. Для улучшения качества изначального ранжирования вектора наименований товаров будут усреднены с топ-5 самых релевантных товару запросов.

In [1]:
import sys
sys.path.append('src/')

In [2]:
from src.tokenizer import Tokenizer
from src.hnsw_index import HNSWIndex

In [3]:
from gensim.models import FastText
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd
import numpy as np
from itertools import groupby
from tqdm.auto import tqdm

tqdm.pandas()

## 1. Data loading

In [4]:
tokenizer = Tokenizer()

In [5]:
model = FastText.load('models/lm')

In [6]:
def vectorize(sentence):
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) == 0:
        return np.array([])
    return np.mean([model.wv[x] for x in tokens], axis=0)

In [9]:
data = pd.read_csv('data/relevance_data.csv')
data

,keyword,item_id,relevance
0,0 000 1 1400 2 673 edition k karcher universal...,100001322597,0.405465
1,0 000 1 1400 2 673 edition k karcher universal...,100001325223,0.693147
2,0 000 1 1400 2 673 edition k karcher universal...,100013196777,0.405465
3,0 000 1 1400 2 673 edition k karcher universal...,100024448131,3.931826
4,0 000 1 1400 2 673 edition k karcher universal...,100026038206,4.624973
...,...,...,...
1154062,ящик,600002996248,0.055570
1154063,ящик,600003699719,0.693147
1154064,ящик,600009268170,3.931826
1154065,ящик,600011403382,2.397895


Наименования товаров загружены ранее и в открытый доступ не выложены.

In [10]:
item_names = pd.read_parquet('data/item_names.parquet')
item_names

,item_id,item_name
0,600006078311,Кружка DRABS Леброн Джеймс 2
1,100032969753,"Чехол Awog ""Ромашковое поле"" для Xiaomi Redmi ..."
2,100033020143,"Чехол Awog ""Зайчик-бананчик"" для Xiaomi Redmi ..."
3,100033005120,"Чехол Awog ""Бордовые розы фон"" для Xiaomi Redm..."
4,100033014330,"Чехол Awog ""На счастье"" для Motorola Moto Edge..."
...,...,...
39452112,100050658252,Водительский Ева коврик VIMCOVЭR для BMW Х1 F4...
39452113,100050673165,Водительский Ева коврик VIMCOVЭR для TOYOTA CR...
39452114,100050660265,Передние Ева коврики VIMCOVЭR для FORD EXPEDIT...
39452115,100050667056,Водительский Ева коврик VIMCOVЭR для OPEL ASTR...


In [11]:
item_names = item_names.set_index('item_id').item_name.to_dict()

## 2. Vectorization

In [12]:
data = data.sort_values(by=['item_id', 'relevance', 'keyword'], ascending=False)

In [13]:
top_n = 5
index_names, index_ids, index_vectors = [], [], []

for item_id, session in tqdm(groupby(data.itertuples(), key=lambda x: x[2]), total=data.item_id.nunique()):
    session = list(session)[:top_n]
    vectors = [vectorize(x.keyword) for x in session]
    item_name = item_names[item_id]
    item_vector = vectorize(item_name)
    if len(item_vector) == 0:
        continue
    vectors.append(item_vector)
    vector = np.mean(vectors, axis=0)
    if len(vector) == 0:
        continue
    index_vectors.append(vector)
    index_ids.append(item_id)
    index_names.append(item_name)

  0%|          | 0/630615 [00:00<?, ?it/s]

In [14]:
index_names = np.asarray(index_names)
index_ids = np.asarray(index_ids)
index_vectors = np.asarray(index_vectors)

In [15]:
index_vectors.shape

(630580, 100)

In [45]:
np.save('data/index_names.npy', index_names)
np.save('data/index_ids.npy', index_ids)
np.save('data/index_vectors.npy', index_vectors)

## 3. Scaling

Шкалирование для поиска процедура совершенно ненужная, но я предполагаю, что настроить коэффициенты при векторах будет проще, если они будут в одном масштабе и больше нуля.

In [30]:
scaler = StandardScaler()
scaled_vectors = scaler.fit_transform(index_vectors)

In [31]:
constant = np.abs(scaled_vectors.min(axis=0))
scaled_vectors = scaled_vectors[:, ] + constant

In [23]:
np.save('data/scaled_vectors.npy', scaled_vectors)

In [39]:
np.save('data/constant.npy', constant)

In [42]:
with open('models/scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)

## 4. Index training

Непосредственно обучение индекса.

In [24]:
%%time
index = HNSWIndex(
    dim=model.vector_size,
    ef_construction=500,
    M=48,
    ef=100
)
index.train(scaled_vectors)

CPU times: user 24min 35s, sys: 36.1 s, total: 25min 12s
Wall time: 22.1 s


In [33]:
def test_search(query, n_search=100):
    vector = vectorize(query).reshape(1, -1)
    vector = scaler.transform(vector) + constant
    I, D = index.index.knn_query(vector, k=n_search)
    return index_names[I[0]]

In [34]:
test_search('apple iphone 14 pro max')[:10]

array(['Смартфон Apple iPhone 14 Pro Max 128Gb Silver (2sim)',
       'Смартфон Apple iPhone 14 Pro 1024Gb Deep Purple (2sim)',
       'Смартфон Apple iPhone 13 Pro Max 1TB Alpine Green',
       'Смартфон Apple iPhone 14 Pro Max 128Gb Space Black (eSIM)',
       'Смартфон Apple iPhone 13 Pro Max 512Gb Alpine green',
       'Смартфон Apple iPhone 14 Pro Max 512Gb Gold (2sim)',
       'Смартфон Apple iPhone 14 Pro 1024Gb Silver (eSIM)',
       'Смартфон Apple iPhone 14 Pro Max 1024Gb Space Black (eSIM)',
       'Смартфон Apple iPhone 14 Pro Max 1024Gb Deep Purple (2sim)',
       'Смартфон Apple iPhone 14 Pro Max 512Gb Silver (2sim)'],
      dtype='<U90')

In [35]:
test_search('xiaomi смартфон')[:10]

array(['Xiaomi Смартфон Xiaomi Redmi 9A Granite Gray, 2/32GB',
       'Смартфон Xiaomi Redmi 10C 4/128Gb, серый',
       'Смартфон Xiaomi Redmi 9C Redmi 9C 2/32GB Lavender (R36600)',
       'Смартфон Xiaomi Redmi 7A 2/16GB Blue',
       'Смартфон Xiaomi 12 8/256GB Blue (37057)',
       'Смартфон Xiaomi POCO M5 4/128GB Green',
       'Смартфон Xiaomi Poco M5s 4/128Gb, голубой',
       'Смартфон Xiaomi Redmi 8 32GB Onyx Black',
       'Смартфон Xiaomi 13 Pro 12/256Gb White',
       'Смартфон Xiaomi Redmi 10A 3/64GB Graphite Gray'], dtype='<U90')

In [36]:
test_search('молоко резня в деревне')[:10]

array(['Молоко 1,5% ультрапастеризованное 925 мл Домик в деревне БЗМЖ',
       'Молоко Домик в деревне ультрапастеризованное 6%, 12 шт х 0,95 л',
       'Молоко 3,5 - 4,5% коровье пастеризованное 930 мл Домик в деревне Отборное БЗМЖ',
       'Творог Домик в деревне традиционный 9% 340 г',
       'Сливки Домик в Деревне питьевые стерилизованные 10% БЗМЖ 200 мл',
       'Кефир Домик в Деревне 2,5% 270 г',
       'Яйцо куриное Кольцовское С1 10 шт',
       'Молоко 3,7% пастеризованное 930 мл Домик в деревне Отборное',
       'Молоко Домик в деревне ультрапастеризованное 2,5%, 12 шт х 0,950 л',
       'Творог рассыпчатый Домик в Деревне отборный 9% 170 г'],
      dtype='<U90')

Как видно, поиск отрабатывает адекватно.

In [37]:
index.save('models/biased_scaled_index')